In [7]:
import cv2
import torch
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
import pandas as pd
import pytorch_lightning as pl
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
from PIL import Image
%matplotlib inline

## Load the data

In [4]:
from data_loader.utils import CustomImageDataset

In [18]:
test_path = './data/chest_xray/test'
train_path = './data/chest_xray/train'
val_path = './data/chest_xray/val'

In [19]:
img_info = pd.read_csv('img_info.csv')
test_df = img_info.loc[img_info.loc[:, 'data'] == 'Test']
train_df = img_info.loc[img_info.loc[:, 'data'] == 'Train']
val_df = test_df = img_info.loc[img_info.loc[:, 'data'] == 'Validation']

In [20]:
# Apply data augmentaion to solve the imbance problem
normal_images = train_df.loc[train_df.loc[:,'label'] == 0]
normal_images.loc[:,'imbalance'] = 1
train_df = pd.concat([train_df, normal_images, normal_images]).reset_index(drop=True)
print('Percentage of Normal Images in the Training Data: {}'.format(
    len(train_df.loc[train_df.loc[:,'label'] == 0])/len(train_df.loc[:,'label'])))
print('Percentage of Pneumonia Images in the Training Data: {}'.format(
    len(train_df.loc[train_df.loc[:,'label'] == 1])/len(train_df.loc[:,'label'])))

Percentage of Normal Images in the Training Data: 0.5093694606229425
Percentage of Pneumonia Images in the Training Data: 0.4906305393770575


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
transforms = T.Compose(
    [
        T.Resize((224,224), antialias=None, interpolation=InterpolationMode.BICUBIC),
        T.RandomApply([
            T.GaussianBlur(kernel_size=(5,5), sigma=(0.1, 0.2))
        ], p=0.5),
        T.RandomEqualize(),
        T.ToTensor()
    ]
)

tranform_test = T.Compose(
    [
        T.Resize((224,224), antialias=None, interpolation=InterpolationMode.BICUBIC),
        T.ToTensor()
    ]
)
imbalance_transform = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=10),
    T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    T.RandomErasing(p=0.2)
])

In [21]:
training_data = CustomImageDataset(train_df, train_path, transforms=transforms)
val_data = CustomImageDataset(val_df, val_path, transforms=transforms)
test_data = CustomImageDataset(test_df, test_path, transforms=transforms)

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

## Model Setup - more layers

In [9]:
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim

In [23]:
def train(model, criterion, optimizer, train_dataloader, val_dataloader, scheduler=None, EPOCHS=25):
    """
    Train the model and return relevant accuracy metric
    
    Input:
        model: the defined neural network
        criterion: the defined loss func
        optimizer: the defined optimizer
        scheduler: if specified, the learning rate is going to decrease with each epoch
        train_dataloader & val_dataloader: DataLoader object
        EPOCHS: int
    Output:
        model: trained model
        train_losses, train_accuracies, val_losses, val_accuracies: acc metrics
    """

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    learning_rate = []

    for epoch in range(EPOCHS):  # loop over the dataset multiple times

        # TRAIN
        # Make sure gradient tracking is on, and do a pass over the data
        model.train()
        running_loss = 0.0
        running_matched = 0
        for i, data in enumerate(train_dataloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data           # NOTE: depending on how you implemented your dataset class's __getitem__ it could be labels, inputs

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # keep track of the loss
            running_loss += loss.item()

            # CALCULATE ACCURACY METRIC
            _, preds = torch.max(outputs, 1)  # Find out the predicted class with the highest prob
            running_matched += torch.sum(preds == labels.data) # caculate the number of matched labels

        avg_train_loss = running_loss / (i + 1)     # i + 1 gives us the total number of batches in train dataloader
        avg_train_acc = running_matched.double() / len(train_dataloader.dataset)
        train_losses.append(avg_train_loss)
        train_accuracies.append(avg_train_acc)

        # VALIDATE
        # In the validation part, we don't want to keep track of the gradients 
        model.eval()
        running_val_loss = 0.0
        running_val_matched = 0
        with torch.no_grad():
            for i, data in enumerate(val_dataloader):
                inputs, labels = data
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # keep track of the loss
                running_val_loss += loss.item()

                # CALCULATE ACCURACY METRIC
                _, preds = torch.max(outputs, 1)
                running_val_matched += torch.sum(preds == labels.data)

        avg_val_loss = running_val_loss / (i + 1)
        avg_val_acc = running_val_matched.double() / len(val_dataloader.dataset)
        val_losses.append(avg_val_loss)
        val_accuracies.append(avg_val_acc)
        learning_rate.append(optimizer.param_groups[0]['lr'])
        
        if scheduler != None:
            scheduler.step(avg_val_loss) # add a scheduler to reduce the lr

        # Print epoch summary
        print(f"Epoch {epoch+1}/{EPOCHS} | "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f} | "
              f"learning rate: {optimizer.param_groups[0]['lr']:.4f}")
        
        
    return model, train_losses, train_accuracies, val_losses, val_accuracies, learning_rate


def test(model, test_dataloader):
    """
    Test the data and return test accuracy
    """

    running_test_matched = 0
    true_labels = []
    predicted_labels = []
    
    with torch.no_grad():
        for i, data in enumerate(test_dataloader):
            inputs, labels = data
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim = 1)
            true_labels.extend(labels.numpy())
            predicted_labels.extend(preds.numpy())
            running_test_matched += torch.sum(preds == labels.data)
    
        test_acc = running_test_matched.double() / len(test_dataloader.dataset)
        precision = precision_score(true_labels, predicted_labels)
        recall = recall_score(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels)

        print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 score: {f1:.2f}")

    return test_acc


In [27]:
import json

def write_model_and_results(model, 
                            train_losses, 
                            train_accuracies, 
                            val_losses, 
                            val_accuracies,
                            learning_rate,
                            path):
    """
    Save the model and results
    
    Inputs:
        path: a list of str containing the path for model and the path for results
    """

    train_acc = [float(i) for i in train_accuracies]
    val_acc = [float(i) for i in val_accuracies]

    result = {'train_losses': train_losses,
             'train_accuracies': train_acc,
             'val_losses': val_losses,
             'val_accuracies': val_acc,
             'learning_rates': learning_rate}
    
    # save model
    torch.save(model.state_dict(), path[0])
    # save results
    with open(path[1], 'w') as f:
        f.write(json.dumps(result))
        

### Sample Model Construction

Here shows the training precess for the CNN with ReLU as the activation function and Negative Loss Likelihood as the loss function.

In [28]:
class CustomNeuralNetwork_relu(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super().__init__()
        
        self.LeNet = nn.Sequential(     
            # convolutional layers            
            nn.Sequential(                                            # FIRST LAYER: (INPUT LAYER)
              nn.Conv2d(3, 6, kernel_size=3, stride=1, padding=0),    # CONVOLUTION 
              nn.BatchNorm2d(6),
              nn.ReLU(),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            nn.Sequential(                                            # SECOND LAYER: HIDDEN LAYER 1
              nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=0),   # CONVOLUTION 
              nn.BatchNorm2d(16),
              nn.ReLU(),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            nn.Sequential(                                            # THIRD LAYER: HIDDEN LAYER 2
              nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0),  # CONVOLUTION 
              nn.BatchNorm2d(32),
              nn.ReLU(),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            nn.Sequential(                                            # FOURTH LAYER: HIDDEN LAYER 3
              nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),  # CONVOLUTION 
              nn.BatchNorm2d(64),
              nn.ReLU(),
              nn.Dropout(p=dropout_rate),
              nn.MaxPool2d(kernel_size = 2, stride = 2)),             # POOLING
            # fully connected layers
            nn.Flatten(),
            # output layer
            nn.Linear(64 * 12 * 12, 2)                                # OUTPUT LAYER
        )

    def forward(self, x):
        out = self.LeNet(x)
        return F.log_softmax(out, dim=1)

In [29]:
model_relu = CustomNeuralNetwork_relu()

In [30]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model_relu.parameters(), lr=0.001, momentum=0.9)

# Inspired by: https://hasty.ai/docs/mp-wiki/scheduler/reducelronplateau
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.1, 
                                                 patience=5, 
                                                 threshold=0.0001, 
                                                 threshold_mode='rel', 
                                                 cooldown=0, 
                                                 min_lr=0, 
                                                 eps=1e-08)

In [ ]:
model_relu, train_losses, train_accuracies, val_losses, val_accuracies, learning_rate = train(model_relu, 
                                                                                              criterion, 
                                                                                              optimizer,
                                                                                              train_dataloader, 
                                                                                              val_dataloader, 
                                                                                              scheduler,
                                                                                              EPOCHS=25)

In [26]:
test(model_relu, test_dataloader)

Test accuracy: 0.7500


tensor(0.7500, dtype=torch.float64)

In [37]:
path = ['model_relu_NLL.PT', 'result_relu_NLL.txt']
write_model_and_results(model_2, train_losses, train_accuracies, val_losses, val_accuracies, path)